# RLVR: Teaching an LLM to Play "Guess the Number"

A fun multi-turn RL demo where we teach a model to play the classic guessing game!

**The Game:**
- Secret number between 0 and 1024
- Model gets 10 guesses
- After each guess: "Too high", "Too low", or "Correct!"
- Optimal strategy: Binary search (can always win in 10 guesses since 2^10 = 1024)

**Why this is interesting:**
- Models often DON'T do perfect binary search
- Multi-turn reasoning is hard
- We can watch the model learn to be more strategic!

---

## What is Tinker?

**Tinker** is a training API that separates concerns:

| You Handle (CPU) | Tinker Handles (GPUs) |
|------------------|----------------------|
| Training loop logic | Distributed training |
| Data/environment | GPU orchestration |
| Reward functions | Forward/backward passes |
| When to update | Gradient accumulation |

```
┌─────────────────────────────────────────────────────────────────┐
│                        YOUR MACHINE (CPU)                       │
│  ┌─────────────┐    ┌─────────────┐    ┌─────────────────────┐  │
│  │   Game      │───→│  Reward     │───→│  Training Loop      │  │
│  │ Environment │    │  Function   │    │  (this notebook!)   │  │
│  └─────────────┘    └─────────────┘    └──────────┬──────────┘  │
│                                                   │ API calls   │
└───────────────────────────────────────────────────┼─────────────┘
                                                    ▼
┌─────────────────────────────────────────────────────────────────┐
│                      TINKER CLOUD (GPUs)                        │
│  ┌─────────────┐    ┌─────────────┐    ┌─────────────────────┐  │
│  │  Sampling   │    │  Forward/   │    │  Optimizer          │  │
│  │  (generate) │    │  Backward   │    │  (update weights)   │  │
│  └─────────────┘    └─────────────┘    └─────────────────────┘  │
└─────────────────────────────────────────────────────────────────┘
```

## What is RLVR?

**Reinforcement Learning with Verifiable Rewards** - RL where we can programmatically check if the answer is correct:

| RLHF (Human Feedback) | RLVR (Verifiable Rewards) |
|----------------------|---------------------------|
| Needs human labelers | Automated reward function |
| Expensive & slow | Fast & scalable |
| Subjective | Objective ground truth |
| "Is this helpful?" | "Did you win the game?" |

## Part 1: Setup

In [1]:
import os
import re
import random
from dataclasses import dataclass
from dotenv import load_dotenv

import tinker
from tinker import types
from tinker.types.tensor_data import TensorData
import torch
from transformers import AutoTokenizer
from tqdm.notebook import tqdm

# Load API key from .env
load_dotenv()
assert os.getenv("TINKER_API_KEY"), "Please set TINKER_API_KEY in your .env file"

print("Setup complete!")

Setup complete!


### 🔑 Key Imports Explained

```python
import tinker                          # Main Tinker SDK
from tinker import types               # Type definitions (Datum, SamplingParams, etc.)
from tinker.types.tensor_data import TensorData  # For passing tensors to API
```

**Tinker's Core Types:**
- `ServiceClient` - Entry point, creates training/sampling clients
- `TrainingClient` - Manages model weights, runs forward/backward passes
- `SamplingClient` - Generates text from current model weights
- `Datum` - A single training example (input + loss function inputs)
- `ModelInput` - Tokenized prompt for the model

In [2]:
# Configuration
MODEL_NAME = "Qwen/Qwen3-4B-Instruct-2507"

LORA_RANK = 32
LEARNING_RATE = 4e-5
BATCH_SIZE = 4           # Games per batch
GROUP_SIZE = 4           # Parallel attempts per game
MAX_TOKENS = 32          # Max tokens per guess
N_TRAINING_BATCHES = 5
MAX_GUESSES = 10         # Guesses allowed per game
NUMBER_RANGE = 1024      # Guess 0 to NUMBER_RANGE-1

print(f"Model: {MODEL_NAME}")
print(f"Game: Guess a number 0-{NUMBER_RANGE-1} in {MAX_GUESSES} tries")
print(f"Optimal strategy: Binary search (always wins since 2^{MAX_GUESSES} = {2**MAX_GUESSES})")

Model: Qwen/Qwen3-4B-Instruct-2507
Game: Guess a number 0-1023 in 10 tries
Optimal strategy: Binary search (always wins since 2^10 = 1024)


### 📊 Hyperparameters Explained

| Parameter | Value | Why? |
|-----------|-------|------|
| `LORA_RANK=32` | LoRA adapter rank | Lower = fewer params, faster. Higher = more expressive |
| `LEARNING_RATE=4e-5` | ~10x higher than full fine-tuning | LoRA needs higher LR since fewer params are updated |
| `GROUP_SIZE=4` | Games per secret number | For GRPO-style variance reduction (compare wins vs losses on same problem) |
| `BATCH_SIZE=4` | Different secret numbers per batch | More diversity in training signal |

**What is LoRA?**

Instead of updating all model weights, LoRA adds small "adapter" matrices:

```
Original:     W (frozen, 4B params)
LoRA:         W + A·B (A and B are small, ~1M params)
                   ↑
            rank=32 controls size
```

This makes training 10-100x more efficient!

In [3]:
# Initialize tokenizer and Tinker client
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
service_client = tinker.ServiceClient()

EOS_TOKEN_ID = tokenizer.eos_token_id
print(f"Connected to Tinker!")

Connected to Tinker!


### 🔗 Tinker Client Hierarchy

```
ServiceClient (entry point)
    │
    ├──▶ create_lora_training_client()
    │         │
    │         ▼
    │    TrainingClient
    │         │
    │         ├──▶ forward_backward()     # Compute gradients
    │         ├──▶ optim_step()           # Update weights  
    │         └──▶ save_weights_and_get_sampling_client()
    │                   │
    │                   ▼
    │              SamplingClient
    │                   │
    │                   └──▶ sample()     # Generate text
    │
    └──▶ create_sampling_client()         # For inference-only
```

**The key insight:** Training and sampling are separate concerns. You get a new `SamplingClient` after each weight update to sample from the updated model.

## Part 2: The Guessing Game Environment

### Multi-Turn RL vs Single-Turn

Most RLVR examples are **single-turn**: one prompt → one response → one reward.

This game is **multi-turn**: the model takes multiple actions before getting a reward!

```
┌──────────────────────────────────────────────────────────────────┐
│                     SINGLE-TURN RLVR                             │
│                                                                  │
│   Prompt ──→ Model ──→ Response ──→ Reward                       │
│   "2+2=?"      │       "4"           ✓ 1.0                       │
│                │                                                 │
└────────────────┼─────────────────────────────────────────────────┘
                 │
┌────────────────┼────────────────────────────────────────────────┐
│                ▼    MULTI-TURN RLVR (This Demo!)                │
│                                                                 │
│   Turn 1: "Guess?" ──→ "512"  ──→ "Too low"                     │
│   Turn 2: "Guess?" ──→ "768"  ──→ "Too high"                    │
│   Turn 3: "Guess?" ──→ "640"  ──→ "Correct!" ──→ Reward: 1.0    │
│                                                                 │
│   All 3 actions get credit for the win!                         │
└─────────────────────────────────────────────────────────────────┘
```

### The Challenge: Credit Assignment

When we win, which guess deserves credit?
- The final correct guess? 
- The earlier guesses that narrowed the range?
- **Answer: All of them!** We assign the same reward to every action in a winning trajectory.

In [4]:
SYSTEM_PROMPT = f"""You are playing a number guessing game. 
- I'm thinking of a number between 0 and {NUMBER_RANGE-1}
- You have {MAX_GUESSES} guesses to find it
- After each guess, I'll tell you if it's too high, too low, or correct
- Output your guess in the format: Guess: <number>
- Just output the guess, nothing else."""

@dataclass
class GameState:
    """Tracks the state of a guessing game."""
    secret: int
    guesses: list  # List of (guess, feedback) tuples
    won: bool = False
    
    def is_done(self) -> bool:
        return self.won or len(self.guesses) >= MAX_GUESSES
    
    def get_reward(self) -> float:
        return 1.0 if self.won else 0.0

def parse_guess(response: str) -> int | None:
    """Extract the guessed number from model response."""
    # Look for "Guess: <number>" pattern
    match = re.search(r'Guess:\s*(\d+)', response)
    if match:
        return int(match.group(1))
    # Fallback: find any number
    numbers = re.findall(r'\d+', response)
    if numbers:
        return int(numbers[0])
    return None

def get_feedback(guess: int, secret: int) -> tuple[str, bool]:
    """Return feedback and whether the guess was correct."""
    if guess == secret:
        return "Correct! You win!", True
    elif guess < secret:
        return "Too low.", False
    else:
        return "Too high.", False

def build_conversation(game: GameState) -> list[dict]:
    """Build the conversation history for the model."""
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    
    for guess, feedback in game.guesses:
        messages.append({"role": "assistant", "content": f"Guess: {guess}"})
        messages.append({"role": "user", "content": feedback})
    
    return messages

def build_model_input(game: GameState) -> types.ModelInput:
    """Build tokenized model input from game state."""
    messages = build_conversation(game)
    prompt_text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    tokens = tokenizer.encode(prompt_text, add_special_tokens=False)
    return types.ModelInput.from_ints(tokens=tokens)

print("Game environment defined!")
print(f"\nExample system prompt:\n{SYSTEM_PROMPT[:200]}...")

Game environment defined!

Example system prompt:
You are playing a number guessing game. 
- I'm thinking of a number between 0 and 1023
- You have 10 guesses to find it
- After each guess, I'll tell you if it's too high, too low, or correct
- Output...


## Part 3: Play a Demo Game (Before Training)

In [5]:
# Create training client
training_client = service_client.create_lora_training_client(
    base_model=MODEL_NAME,
    rank=LORA_RANK
)
print(f"Created training client with LoRA rank {LORA_RANK}")

# Get initial sampling client
initial_sampler = training_client.save_weights_and_get_sampling_client(name="initial")
print("Ready to play!")

Created training client with LoRA rank 32
Ready to play!


### 🎮 Key API: `sample()`

The sampling client generates text from the model:

```python
result = sampling_client.sample(
    prompt=model_input,           # Tokenized input (ModelInput)
    num_samples=1,                # How many completions to generate
    sampling_params=SamplingParams(
        max_tokens=32,            # Max tokens to generate
        temperature=0.0,          # 0=greedy, >0=random
        stop=[EOS_TOKEN_ID]       # Stop tokens
    )
).result()                        # .result() waits for completion

# Access the generated tokens and their log probabilities
tokens = result.sequences[0].tokens      # List[int]
logprobs = result.sequences[0].logprobs  # List[float] - needed for training!
```

**Why do we need logprobs?**

For policy gradient training, we need to know how likely the model thought each token was. This is used in the importance sampling loss to correct for distribution shift.

In [6]:
def play_game(sampling_client, secret: int, verbose: bool = True) -> GameState:
    """Play a complete guessing game and return the final state."""
    game = GameState(secret=secret, guesses=[])
    
    if verbose:
        print(f"\n🎯 Secret number: {secret}")
        print("-" * 40)
    
    while not game.is_done():
        # Get model's guess
        model_input = build_model_input(game)
        result = sampling_client.sample(
            prompt=model_input,
            num_samples=1,
            sampling_params=types.SamplingParams(
                max_tokens=MAX_TOKENS,
                temperature=0.0,
                stop=[EOS_TOKEN_ID] if EOS_TOKEN_ID else []
            )
        ).result()
        
        response = tokenizer.decode(result.sequences[0].tokens, skip_special_tokens=True)
        guess = parse_guess(response)
        
        if guess is None:
            if verbose:
                print(f"Turn {len(game.guesses)+1}: Failed to parse guess from '{response[:50]}'")
            game.guesses.append((0, "Invalid guess. Try again."))
            continue
        
        feedback, correct = get_feedback(guess, secret)
        game.guesses.append((guess, feedback))
        game.won = correct
        
        if verbose:
            status = "✓" if correct else "→"
            print(f"Turn {len(game.guesses)}: Guess {guess} {status} {feedback}")
    
    if verbose:
        print("-" * 40)
        print(f"Result: {'🎉 WON' if game.won else '❌ LOST'} in {len(game.guesses)} turns")
    
    return game

# Play a demo game
print("=" * 50)
print("DEMO GAME (Before Training)")
print("=" * 50)
demo_game = play_game(initial_sampler, secret=640)

DEMO GAME (Before Training)

🎯 Secret number: 640
----------------------------------------
Turn 1: Guess 512 → Too low.
Turn 2: Guess 768 → Too high.
Turn 3: Guess 640 ✓ Correct! You win!
----------------------------------------
Result: 🎉 WON in 3 turns


In [7]:
def evaluate_model(sampling_client, n_games: int = 20) -> float:
    """Evaluate win rate over multiple games."""
    wins = 0
    total_guesses = 0
    
    random.seed(42)
    secrets = [random.randint(0, NUMBER_RANGE-1) for _ in range(n_games)]
    
    for secret in tqdm(secrets, desc="Evaluating"):
        game = play_game(sampling_client, secret, verbose=False)
        wins += 1 if game.won else 0
        total_guesses += len(game.guesses)
    
    win_rate = wins / n_games
    avg_guesses = total_guesses / n_games
    
    print(f"\nResults over {n_games} games:")
    print(f"  Win rate: {wins}/{n_games} = {win_rate:.1%}")
    print(f"  Avg guesses: {avg_guesses:.1f}")
    
    return win_rate

print("Evaluating BEFORE training...")
before_win_rate = evaluate_model(initial_sampler, n_games=20)

Evaluating BEFORE training...


Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]


Results over 20 games:
  Win rate: 2/20 = 10.0%
  Avg guesses: 10.0


## Part 4: RLVR Training

### The Training Loop

```
┌─────────────────────────────────────────────────────────────────────┐
│                    RLVR TRAINING LOOP                               │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  ┌──────────────────────────────────────────────────────────────┐   │
│  │ 1. GET SAMPLING CLIENT                                       │   │
│  │    sampling_client = training_client                         │   │
│  │        .save_weights_and_get_sampling_client()               │   │
│  └──────────────────────────────────────────────────────────────┘   │
│                              │                                      │
│                              ▼                                      │
│  ┌──────────────────────────────────────────────────────────────┐   │
│  │ 2. COLLECT TRAJECTORIES                                      │   │
│  │    For each game:                                            │   │
│  │      - Play until win/lose                                   │   │
│  │      - Record (prompt, response, logprobs) for each turn     │   │
│  │      - Get final reward (1.0 = win, 0.0 = lose)              │   │
│  └──────────────────────────────────────────────────────────────┘   │
│                              │                                      │
│                              ▼                                      │
│  ┌──────────────────────────────────────────────────────────────┐   │
│  │ 3. COMPUTE ADVANTAGES (GRPO-style)                           │   │
│  │    advantage = reward - mean(rewards)                        │   │
│  │                                                              │   │
│  │    Win:  advantage = 1.0 - 0.5 = +0.5  (reinforce!)          │   │
│  │    Lose: advantage = 0.0 - 0.5 = -0.5  (discourage!)         │   │
│  └──────────────────────────────────────────────────────────────┘   │
│                              │                                      │
│                              ▼                                      │
│  ┌──────────────────────────────────────────────────────────────┐   │
│  │ 4. BUILD TRAINING DATA                                       │   │
│  │    datum = Datum(                                            │   │
│  │        model_input = prompt + response,                      │   │
│  │        loss_fn_inputs = {                                    │   │
│  │            "target_tokens": response tokens,                 │   │
│  │            "logprobs": sampling logprobs,                    │   │
│  │            "advantages": reward signal                       │   │
│  │        }                                                     │   │
│  │    )                                                         │   │
│  └──────────────────────────────────────────────────────────────┘   │
│                              │                                      │
│                              ▼                                      │
│  ┌──────────────────────────────────────────────────────────────┐   │
│  │ 5. UPDATE MODEL                                              │   │
│  │    training_client.forward_backward(datums, loss_fn=...)     │   │
│  │    training_client.optim_step(adam_params)                   │   │
│  └──────────────────────────────────────────────────────────────┘   │
│                              │                                      │
│                              ▼                                      │
│                         REPEAT                                      │
└─────────────────────────────────────────────────────────────────────┘
```

### Key API: `forward_backward()`

This is where the magic happens! Tinker computes gradients on GPUs:

```python
training_client.forward_backward(
    datums,                          # List of training examples
    loss_fn="importance_sampling"    # Policy gradient loss
)
```

**Loss function options:**
| Loss | Use Case |
|------|----------|
| `"cross_entropy"` | Supervised learning (SFT) |
| `"importance_sampling"` | Basic policy gradient (REINFORCE) |
| `"ppo"` | PPO with clipping |

### Key API: `optim_step()`

Updates the model weights using accumulated gradients:

```python
training_client.optim_step(
    AdamParams(learning_rate=4e-5, beta1=0.9, beta2=0.95)
)
```

In [8]:
# Training hyperparameters
adam_params = types.AdamParams(
    learning_rate=LEARNING_RATE,
    beta1=0.9,
    beta2=0.95,
    eps=1e-8
)

sampling_params = types.SamplingParams(
    max_tokens=MAX_TOKENS,
    temperature=0.7,  # Exploration during training
    stop=[EOS_TOKEN_ID] if EOS_TOKEN_ID else []
)

training_metrics = []
print("Training params configured!")

Training params configured!


In [9]:
def collect_game_trajectory(sampling_client, secret: int) -> tuple[list, float]:
    """
    Play a game and collect all (prompt, response, logprobs) for training.
    Returns (trajectory, reward)
    """
    game = GameState(secret=secret, guesses=[])
    trajectory = []  # List of (model_input, tokens, logprobs)
    
    while not game.is_done():
        model_input = build_model_input(game)
        
        result = sampling_client.sample(
            prompt=model_input,
            num_samples=1,
            sampling_params=sampling_params
        ).result()
        
        seq = result.sequences[0]
        response = tokenizer.decode(seq.tokens, skip_special_tokens=True)
        guess = parse_guess(response)
        
        # Store trajectory data
        trajectory.append((model_input, seq.tokens, seq.logprobs))
        
        if guess is None:
            game.guesses.append((0, "Invalid guess."))
            continue
        
        feedback, correct = get_feedback(guess, secret)
        game.guesses.append((guess, feedback))
        game.won = correct
    
    return trajectory, game.get_reward()

print("Trajectory collector defined!")

Trajectory collector defined!


In [10]:
print("Starting RLVR training...")
print("=" * 50)

for batch_idx in range(N_TRAINING_BATCHES):
    # Get sampling client for current weights
    sampling_client = training_client.save_weights_and_get_sampling_client(
        name=f"step_{batch_idx}"
    )
    
    # Generate random secrets for this batch
    secrets = [random.randint(0, NUMBER_RANGE-1) for _ in range(BATCH_SIZE)]
    
    # Collect trajectories for multiple games
    all_trajectories = []  # (trajectory, reward) pairs
    
    for secret in tqdm(secrets, desc=f"Batch {batch_idx+1}/{N_TRAINING_BATCHES}"):
        # Play GROUP_SIZE games with same secret for variance reduction
        for _ in range(GROUP_SIZE):
            traj, reward = collect_game_trajectory(sampling_client, secret)
            all_trajectories.append((traj, reward))
    
    # Compute rewards and advantages
    rewards = [r for _, r in all_trajectories]
    mean_reward = sum(rewards) / len(rewards)
    
    # Build training datums from trajectories
    datums = []
    
    for trajectory, reward in all_trajectories:
        advantage = reward - mean_reward
        
        # Skip if no learning signal
        if advantage == 0:
            continue
        
        # Create datum for each turn in the trajectory
        for model_input, tokens, logprobs in trajectory:
            if not tokens or logprobs is None:
                continue
            
            ob_len = model_input.length - 1
            
            # Build full sequence
            full_input = model_input.append(
                types.EncodedTextChunk(tokens=tokens[:-1] if len(tokens) > 1 else tokens)
            )
            
            # Pad tensors
            target_tokens = [0] * ob_len + list(tokens)
            padded_logprobs = [0.0] * ob_len + list(logprobs)
            padded_advantages = [0.0] * ob_len + [advantage] * len(tokens)
            
            # Truncate to match
            seq_len = full_input.length
            
            datum = types.Datum(
                model_input=full_input,
                loss_fn_inputs={
                    "target_tokens": TensorData.from_torch(torch.tensor(target_tokens[:seq_len])),
                    "logprobs": TensorData.from_torch(torch.tensor(padded_logprobs[:seq_len])),
                    "advantages": TensorData.from_torch(torch.tensor(padded_advantages[:seq_len])),
                }
            )
            datums.append(datum)
    
    # Training step
    if datums:
        fwd_bwd_future = training_client.forward_backward(datums, loss_fn="importance_sampling")
        optim_future = training_client.optim_step(adam_params)
        fwd_bwd_future.result()
        optim_future.result()
    
    # Log metrics
    training_metrics.append({
        "batch": batch_idx,
        "win_rate": mean_reward,
        "n_datums": len(datums)
    })
    
    print(f"Batch {batch_idx+1}: Win Rate = {mean_reward:.1%} | Training samples = {len(datums)}")

print("=" * 50)
print("Training complete!")

Starting RLVR training...


Batch 1/5:   0%|          | 0/4 [00:00<?, ?it/s]

Batch 1: Win Rate = 31.2% | Training samples = 154


Batch 2/5:   0%|          | 0/4 [00:00<?, ?it/s]

Batch 2: Win Rate = 50.0% | Training samples = 140


...

Batch 3: Win Rate = 0.0% | Training samples = 0


Batch 4/5:   0%|          | 0/4 [00:00<?, ?it/s]

Batch 4: Win Rate = 56.2% | Training samples = 141


Batch 5/5:   0%|          | 0/4 [00:00<?, ?it/s]

Batch 5: Win Rate = 25.0% | Training samples = 160
Training complete!


## Part 5: Evaluate After Training

In [11]:
# Get final sampling client
final_sampler = training_client.save_weights_and_get_sampling_client(name="final")

print("Evaluating AFTER training...")
after_win_rate = evaluate_model(final_sampler, n_games=20)

Evaluating AFTER training...


Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]


Results over 20 games:
  Win rate: 3/20 = 15.0%
  Avg guesses: 10.0


In [12]:
# Play a demo game with trained model
print("\n" + "=" * 50)
print("DEMO GAME (After Training)")
print("=" * 50)
demo_game_after = play_game(final_sampler, secret=640)


DEMO GAME (After Training)

🎯 Secret number: 640
----------------------------------------
Turn 1: Guess 512 → Too low.
Turn 2: Guess 768 → Too high.
Turn 3: Guess 640 ✓ Correct! You win!
----------------------------------------
Result: 🎉 WON in 3 turns


In [13]:
# Summary
print("\n" + "=" * 50)
print("RLVR Training Summary: Guess the Number")
print("=" * 50)
print(f"Model: {MODEL_NAME}")
print(f"Game: Guess 0-{NUMBER_RANGE-1} in {MAX_GUESSES} tries")
print(f"Training: {N_TRAINING_BATCHES} batches")
print(f"\nWin Rate:")
print(f"  Before: {before_win_rate:.1%}")
print(f"  After:  {after_win_rate:.1%}")
improvement = after_win_rate - before_win_rate
print(f"  Change: {improvement:+.1%} {'🎉' if improvement > 0 else ''}")
print("\nTraining Progress:")
for m in training_metrics:
    bar = "█" * int(m['win_rate'] * 20)
    print(f"  Batch {m['batch']+1}: {m['win_rate']:.1%} {bar}")


RLVR Training Summary: Guess the Number
Model: Qwen/Qwen3-4B-Instruct-2507
Game: Guess 0-1023 in 10 tries
Training: 5 batches

Win Rate:
  Before: 10.0%
  After:  15.0%
  Change: +5.0% 🎉

Training Progress:
  Batch 1: 31.2% ██████
  Batch 2: 50.0% ██████████
  Batch 3: 0.0% 
  Batch 4: 56.2% ███████████
  Batch 5: 25.0% █████


## Summary

### Tinker API Recap

```
┌────────────────────────────────────────────────────────────────┐
│                     TINKER API CHEATSHEET                      │
├────────────────────────────────────────────────────────────────┤
│                                                                │
│  SETUP                                                         │
│  ─────                                                         │
│  service_client = tinker.ServiceClient()                       │
│  training_client = service_client.create_lora_training_client( │
│      base_model="Qwen/Qwen3-4B-Instruct-2507",                 │
│      rank=32                                                   │
│  )                                                             │
│                                                                │
│  SAMPLING                                                      │
│  ────────                                                      │
│  sampler = training_client.save_weights_and_get_sampling_client│
│  result = sampler.sample(prompt, num_samples=8, ...)           │
│  tokens = result.sequences[0].tokens                           │
│  logprobs = result.sequences[0].logprobs                       │
│                                                                │
│  TRAINING                                                      │
│  ────────                                                      │
│  datum = Datum(model_input, loss_fn_inputs={                   │
│      "target_tokens": ...,                                     │
│      "logprobs": ...,                                          │
│      "advantages": ...                                         │
│  })                                                            │
│  training_client.forward_backward(datums, loss_fn="ppo")       │
│  training_client.optim_step(AdamParams(lr=4e-5))               │
│                                                                │
└────────────────────────────────────────────────────────────────┘
```

### What We Learned

| Concept | What It Means |
|---------|--------------|
| **RLVR** | RL with programmatic rewards (no humans needed!) |
| **Multi-turn RL** | Collect full trajectories, assign sparse rewards |
| **GRPO** | Compare wins vs losses on same problem for low-variance gradients |
| **LoRA** | Efficient fine-tuning by only updating small adapter matrices |
| **Importance Sampling** | Correct for policy drift during training |

### Why This Matters

1. **Verifiable rewards scale** - No human labelers needed
2. **Multi-turn reasoning** - Teach models to plan ahead  
3. **Game playing** - Natural testbed for strategic behavior
4. **Binary search** - Optimal strategy the model can discover through RL

### Next Steps

- Try the [Tinker Cookbook](https://github.com/thinking-machines-lab/tinker-cookbook) for more recipes
- Explore GSM8K math reasoning
- Try code generation with unit test rewards
- Build your own RL environment!